In [6]:
import tensorflow as tf
import torch

# CUDA 빌드 여부 확인 (TensorFlow 2.x)
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Built with CUDA: True
Using device: cuda


In [7]:
import transformers
# Llama 3 모델을 사용하기 위한 라이브러리 설치
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'Backend' from 'torch._C._distributed_c10d' (unknown location)